# Creating, Planting, and Finding Signals Within Noise
Elizabeth Kapelevich

In [2]:
import math
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import signals_and_noise as san

In [12]:
Time1, Position1 = np.loadtxt("mock_data_challenge_1.dat").T
Time2, Position2 = np.loadtxt("mock_data_challenge_2.dat").T

In [ ]:
%%timeit
san.fs_search(0.012, 2, 20, [Time1, Position1], 1.5, 8, 0.012, 2.0, 20, 10, 2.0, 10, 5.0)

In [ ]:
%%timeit
san.fs_search(0.012, 2, 20, [Time2, Position2], 0.5, 4, 0.012, 2.0, 20, 0.5, 2.0, 10, 0.5)

## Meshgrid For-Loop Assignment: 
Meshgrid creates matrices that have proportionate sizes.
* Within the `template` function, create a meshgrid that will allow many different values of frequencies that can be used by many different values of the `S` equation.
* Using a for-loop, evaluate ` S ` over values of ` frequency ` and ` time_stamps ` so that the output is an array of ` S ` values.
* Once that is complete, now go through ` integrator ` function and ` cross_correlation ` function and update the code so that the frequency parameter can be properly passed as an array and be properly calculated. Using a meshgrid will help.

In [86]:
def template(a, f, sigma, t_0, t_duration, data_time_stamps, ad_hoc_grid = 10000):
    
    """
    This function will take as input the frequency, standard
    deviation, initial time, duration time, and interval
    of time between each value.
    
    INPUT:
    ------
    A : amplitude
    f : array of frequencies
    sigma : array of standard deviations
    t_0 : start time of a signal
    t_duration : duration of time for the signal
    data_time_stamps : time stamps from the data
    
    RETURNS:
    --------
    A template that will match the given signal with 
    the proper frequency and standard deviation using
    zero-padding.

    """
    
    del_T_data = np.diff(data_time_stamps)[0]                       # finding interval between values
    t_end = t_duration + t_0
    time_stamps = np.linspace(t_0, t_end + del_T_data, ad_hoc_grid) # creating array of values with random step-size
    
    t_mean = np.mean(time_stamps)
    t_stamps, frequency = np.meshgrid(time_stamps, f)                       # setting up a matrix for interpolation

    S = a*np.sin(2*np.pi*frequency*t_stamps)*np.exp((-(t_stamps - t_mean)**2)/(2*sigma)) # evaluation in ad-hoc grid
    # print(len(S[0])) -> how many measurements of frequency
    print(len(f))
    print(len(time_stamps))
    all_templates = []                                                   # empty list that will store Template values
    
    for i in range(len(f)):
        s = S[i] # not sure how to set this up
        interpolant = interpolate.interp1d(time_stamps, s)             # prepping interpolation of signal in time-stamps of data-grid
        Template = interpolant(data_time_stamps[(data_time_stamps >= t_0)*(data_time_stamps <= t_end)]) # interpolating using bounds

        template_series = np.zeros_like(data_time_stamps)           # template time-stamps same length as data time-stamps

        index_prefix = template_series[data_time_stamps < t_0]      # zero-padding template on the left side
        index_suffix = template_series[data_time_stamps > t_end]    # zero-padding template on the right side

        closest_to_t0 = data_time_stamps[np.argmin(np.abs(t_0 - data_time_stamps))]     # approximating minimum value near beginning of template
        closest_to_tend = data_time_stamps[np.argmin(np.abs(t_end - data_time_stamps))] # approximating minimum value near ending of template
        
        result = np.hstack((index_prefix, Template, index_suffix))  # horizontally stacking zero-padded values around template
        all_templates.append(result)                                     # appending all values into Template
        
    return Template

In [87]:
a = 1.5
f = np.array([1, 2, 3])
sigma = 0.1
t_0 = 5
t_duration = 2
data_time_stamps = Time1

In [88]:
my_temp = template(a, f, sigma, t_0, t_duration, data_time_stamps)

3
10000


In [75]:
a = np.array([1, 2, 3, 4, 5])
b = np.array([10, 20, 30, 40])
i = np.array([10, 20, 30, 40])
j = np.array([1, 2, 3, 4, 5])

In [76]:
A, B = np.meshgrid(a, b)
I, J = np.meshgrid(i, j)
print(A)
print(B)
print(A*B)

[[1 2 3 4 5]
 [1 2 3 4 5]
 [1 2 3 4 5]
 [1 2 3 4 5]]
[[10 10 10 10 10]
 [20 20 20 20 20]
 [30 30 30 30 30]
 [40 40 40 40 40]]
[[ 10  20  30  40  50]
 [ 20  40  60  80 100]
 [ 30  60  90 120 150]
 [ 40  80 120 160 200]]
